# Worldwide Earthquake Events API - Gold Layer Processing

In [ ]:
from pyspark.sql.functions import when, col, udf
from pyspark.sql.types import StringType

import reverse_geocoder as rg

In [ ]:
df = spark.read.table("earthquake_events_silver").filter(col('time') > start_date)

In [ ]:
def get_country_code(lat, lon):
    """
    Récupère le code pays pour une latitude et longitude données.

    Paramètres :
    lat (float ou str) : Latitude du lieu
    lon (float ou str) : Longitude du lieu

    Retourne :
    str : Code du pays, obtenu en utilisant l'API de géocodage inversé

    Exemple :
    >>> get_country_details(48.8588443, 2.2943506)
    'FR'
    """
    coordinates = (float(lat), float(lon))
    return rg.search(coordinates)[0].get('cc')

In [ ]:
# Enregistrer la UDF pour pouvoir l'utiliser dans Spark DataFrames
get_country_code_udf = udf(get_country_code, StringType())

In [ ]:
# Ajouter les colonnes code pays et éventuelle ville
df_with_location = \
                df.\
                    withColumn("country_code", get_country_code_udf(col("latitude"), col("longitude")))

In [ ]:
# Ajouter la classification de significativité
df_with_location_sig_class = \
                            df_with_location.\
                                withColumn('sig_class', 
                                            when(col("sig") < 100, "Low").\
                                            when((col("sig") >= 100) & (col("sig") < 500), "Moderate").\
                                            otherwise("High")
                                            )

In [ ]:
# Ajouter les données à la table gold
df_with_location_sig_class.write.mode('append').saveAsTable('earthquake_events_gold')